# Natural Language Processing with Tensorflow

This notebook contains:
  * Downloading a text dataset
  * Visualizing the data
  * Converting text into numbers using tokenization
  * Turning tokenized text into an embedding
  * Modelling a text data
    * Starting with a baseline (TF-IDF)
    * Building several deep learning text models
      * Dense, LSTM, GRU, Conv1D, Transfer Learning
  * Comparing the performance of each of our models
  * Combining our models into an ensemble
  * Saving and loading a trained model
  * Finding the most wrong predictions

## Checking the GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-937ef8a2-b4c4-0990-6a78-e1e2881e1869)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-02-11 09:40:44--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-02-11 09:40:44 (106 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

# Download a text dataset

In [4]:
# Download data
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

--2023-02-11 09:40:48--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.148.128, 209.85.200.128, 142.250.152.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.148.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2023-02-11 09:40:48 (157 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
# Unzip data
unzip_data("nlp_getting_started.zip")

# Visualizing the dataset

In [6]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac = 1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
# How many samples total
print(f"Total training samples: {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total samples: {len(train_df)+len(test_df)}")

Total training samples: 7613
Total test samples: 3263
Total samples: 10876


In [11]:
# Visualizing some random samples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text: {text}")
  print("---")

Target: 0 (not real disaster)
Text: Zayn just blew up twitter.
---
Target: 1 (real disaster)
Text: @ItsQueenBaby I'm at work it's a bunch of ppl and buses because the train derailed
---
Target: 0 (not real disaster)
Text: @Adanne___ kindly follow back
---
Target: 1 (real disaster)
Text: @robbiewilliams U fkn asswipe playing for Israeli child killers.. The fkn karma police will get U.
---
Target: 1 (real disaster)
Text: We have different moral systems. Mine rejects the mass murder of innocents yours explicitly endorses such behavior. https://t.co/qadRKEJZ9T
---


### Split the data into training and validation sets

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_df_shuffled['text'].to_numpy(), train_df_shuffled['target'].to_numpy(), test_size=0.1, random_state=42)

In [13]:
# Check the sizes of train and val sets
len(X_train), len(X_val), len(y_train), len(y_val)

(6851, 762, 6851, 762)

# Converting Text into Numbers

### Text Vectorization

In [14]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None)

In [15]:
# Find average number of tokens in training Tweets
round(sum([len(i.split()) for i in X_train])/len(X_train))

15

In [16]:
#@title Default title text
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                   output_mode="int",
                                   output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(X_train)

In [18]:
# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(X_train)
print(f"Original text:\n{random_sentence}\nVectorized text: \n{text_vectorizer(random_sentence)}")

Original text:
Just burned the crap out of my grilled cheese sandwich. Sure wish I had a few life skills figured out by now...
Vectorized text: 
[  29  466    2 2147   36    6   13    1 1884 8829  474 1160    8   94
    3]


In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

In [21]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length="max_length",
                             name="embedding_1")

embedding

In [22]:
# Get a random sentence from training set
random_sentence = random.choice(X_train)
print(f"Original text:\n{random_sentence}\nEmbedded version:")
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
#raining #flooding #Florida #TampaBay #Tampa 18 or 19 days. I've lost count 
Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03308056, -0.0258514 , -0.01347691, ..., -0.03574706,
          0.00123221, -0.00241047],
        [ 0.01526823, -0.00646883, -0.01214958, ...,  0.02676501,
         -0.02885267, -0.04517703],
        [ 0.02106586,  0.02289457,  0.04740734, ...,  0.00654104,
          0.02781364, -0.02431784],
        ...,
        [ 0.01625851, -0.01583682,  0.00636747, ...,  0.03171028,
          0.04938394, -0.04434614],
        [ 0.01625851, -0.01583682,  0.00636747, ...,  0.03171028,
          0.04938394, -0.04434614],
        [ 0.01625851, -0.01583682,  0.00636747, ...,  0.03171028,
          0.04938394, -0.04434614]]], dtype=float32)>

In [23]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.03308056, -0.0258514 , -0.01347691,  0.04656169, -0.0437247 ,
        0.04271102,  0.00044273, -0.03157001,  0.00640529, -0.01748576,
       -0.01065675, -0.01325931,  0.03346613, -0.03227892,  0.04141915,
        0.04205138,  0.02364134,  0.0008806 , -0.03089856,  0.02802502,
        0.03973866, -0.03642274,  0.03628773, -0.01460359, -0.01981978,
       -0.03869236, -0.02523896, -0.04396894, -0.02236195, -0.04212712,
       -0.04096161,  0.00559261, -0.02198335, -0.03915991,  0.04526284,
        0.03389959,  0.0327626 ,  0.00430737,  0.0117601 ,  0.04042555,
       -0.0339824 ,  0.00908756, -0.02584334, -0.00256453, -0.02626472,
        0.01099221,  0.03170638, -0.02933301, -0.02704583, -0.04047859,
       -0.00663448, -0.03446575, -0.02460034,  0.03515399,  0.01453333,
        0.01655794,  0.02688767, -0.01327866,  0.01562642, -0.02948608,
        0.00421422,  0.04252727,  0.02298857, -0.03321778, -0.01529136,
        0.008608

# Modelling a text dataset

* **Model 0**: Naive bayes (baseline)
* **Model 1**: Feed-forward neural network (dense model)
* **Model 2**: LSTM model
* **Model 3**: GRU model
* **Model 4**: Bidirectional-LSTM model
* **Model 5**: 1D Convolutional Neural Network
* **Model 6**: Tensorflow Hub Pretrained Feature Extractor
* **Model 7**: Same as model 6 with 10% of training data

### Model 0: Getting a baseline

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data
model_0.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
baseline_score = model_0.score(X_val, y_val)
print(f"Our baseline model achieves the accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves the accuracy of: 79.27%


In [26]:
# Make predictions
baseline_preds = model_0.predict(X_val)
baseline_preds[:20]


array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments
  * Accuracy
  * Precision
  * Recall
  * F1-Score

In [27]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1":model_f1}
  return model_results

In [28]:
# Get baseline results
baseline_results = calculate_results(y_true=y_val, y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A Simple dense model

In [29]:
# Create tensorboard callback
from helper_functions import create_tensorboard_callback

# Create directory to save tensorboard logs
SAVE_DIR = "model_logs"

In [30]:
# Build model with the functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [31]:
# Compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [32]:
# Get summary of the model
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [33]:
# Fit the model
model_1.history = model_1.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=[X_val, y_val],
                              callbacks=[create_tensorboard_callback(dir_name="SAVE_DIR",
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: SAVE_DIR/simple_dense_model/20230211-094055
Epoch 1/5
215/215 [==============================] - 19s 54ms/step - loss: 0.6100 - accuracy: 0.6923 - val_loss: 0.5364 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.4418 - accuracy: 0.8178 - val_loss: 0.4692 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3469 - accuracy: 0.8600 - val_loss: 0.4588 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2853 - accuracy: 0.8923 - val_loss: 0.4637 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2384 - accuracy: 0.9123 - val_loss: 0.4760 - val_accuracy: 0.7874


In [34]:
# Check the results
model_1.evaluate(X_val, y_val)

24/24 [==============================] - 0s 3ms/step - loss: 0.4760 - accuracy: 0.7874


[0.47597330808639526, 0.787401556968689]

In [35]:
embedding.weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[ 0.03448541, -0.03558839,  0.02555773, ...,  0.05110313,
          0.03177845, -0.05058347],
        [ 0.03576934, -0.02776362, -0.01136329, ...,  0.05185858,
         -0.04293076,  0.05562627],
        [-0.00413558, -0.01525933,  0.02917902, ...,  0.02332357,
         -0.00246887,  0.02395228],
        ...,
        [ 0.01640428,  0.00540668, -0.00585657, ...,  0.00142689,
          0.03212271,  0.03632977],
        [ 0.0557108 , -0.0821947 ,  0.05216848, ...,  0.07917666,
         -0.0057567 , -0.04316576],
        [ 0.10618868, -0.07152995,  0.05860833, ...,  0.06857188,
         -0.10077745, -0.09165118]], dtype=float32)>]

In [36]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [37]:
# Make predictions
model_1_pred_probs = model_1.predict(X_val)
model_1_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.4164838 ],
       [0.7455729 ],
       [0.9977216 ],
       [0.10908118],
       [0.10797716],
       [0.93715316],
       [0.9150384 ],
       [0.9931544 ],
       [0.9700474 ],
       [0.25891498]], dtype=float32)

In [38]:
# Turn prediction probabilities into single dimension tensor of floats
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [39]:
# Calculate model_1 metrics
model_1_results = calculate_results(y_true = y_val, y_pred = model_1_preds)
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7914920592553047,
 'recall': 0.7874015748031497,
 'f1': 0.7846966492209201}

### Model 2: LSTM

In [40]:
# Set random seed and create embedding layer
tf.random.set_seed(42)
from tensorflow.keras import layers
model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")

# Create LSTM model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)
print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
# x = layers.Dense(1, activation="sigmoid")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)


In [41]:
# Compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [42]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [43]:
# Fit the model
model_2_history = model_2.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=(X_val, y_val),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20230211-094139
Epoch 1/5
215/215 [==============================] - 13s 42ms/step - loss: 0.5088 - accuracy: 0.7479 - val_loss: 0.4537 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3174 - accuracy: 0.8720 - val_loss: 0.5105 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2194 - accuracy: 0.9164 - val_loss: 0.5883 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1552 - accuracy: 0.9451 - val_loss: 0.6198 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1082 - accuracy: 0.9590 - val_loss: 0.8454 - val_accuracy: 0.7651


In [44]:
# Make prediction on the validation dataset
model_2_pred_probs = model_2.predict(X_val)
model_2_pred_probs.shape, model_2_pred_probs[:10]

24/24 [==============================] - 1s 3ms/step


((762, 1), array([[0.00731559],
        [0.75501204],
        [0.99899334],
        [0.07022057],
        [0.00299372],
        [0.9990152 ],
        [0.9436    ],
        [0.99962306],
        [0.99932504],
        [0.3318296 ]], dtype=float32))

In [45]:
# Round up predictions and reduce it to 1D array
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [46]:
# Calculate LSTM model results
model_2_results = calculate_results(y_val, model_2_preds)
model_2_results

{'accuracy': 76.50918635170603,
 'precision': 0.7675147482403992,
 'recall': 0.7650918635170604,
 'f1': 0.7624764882687336}

### Model 3: GRU

In [47]:
# Set random seed and create embedding layer
tf.random.set_seed(42)
from tensorflow.keras import layers
model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_3")

# Build an RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="GRU")

In [48]:
# Compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [49]:
model_3.summary()

Model: "GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________________

In [50]:
# Fit the model
model_3_history = model_3.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=(X_val, y_val),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"GRU")])

Saving TensorBoard log files to: model_logs/GRU/20230211-094203
Epoch 1/5
215/215 [==============================] - 11s 41ms/step - loss: 0.5244 - accuracy: 0.7319 - val_loss: 0.4542 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3198 - accuracy: 0.8680 - val_loss: 0.4915 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2208 - accuracy: 0.9171 - val_loss: 0.5615 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1628 - accuracy: 0.9416 - val_loss: 0.6064 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1237 - accuracy: 0.9585 - val_loss: 0.6142 - val_accuracy: 0.7690


In [51]:
# Make prediction on validation data
model_3_pred_probs = model_3.predict(X_val)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[0.42440858],
       [0.83296824],
       [0.99715316],
       [0.12658606],
       [0.01395389],
       [0.9900193 ],
       [0.5614211 ],
       [0.9972082 ],
       [0.99769115],
       [0.5041359 ]], dtype=float32)

In [52]:
# Round up the prediction and convert it to 1D array
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [53]:
# Calculate result
model_3_results = calculate_results(y_val, model_3_preds)
model_3_results

{'accuracy': 76.9028871391076,
 'precision': 0.7687139520620504,
 'recall': 0.7690288713910761,
 'f1': 0.7683358548586026}

### Model 4: Bidirectional RNN Model

In [56]:
# Set random seed and create embedding layer
tf.random.set_seed(42)
from tensorflow.keras import layers
model_4_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_4")

# Build a Bidirectional RNN in tensorflow
input = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(input)
x = model_4_embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
output = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(input, output, name="model_4_Bidirectional")

In [57]:
# Compile
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [58]:
# Get a summary of our bidirectional model
model_4.summary()

Model: "model_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_4 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [59]:
# Fit the model
model_4_history = model_4.fit(X_train,
                              y_train,
                              epochs=5,
                              validation_data=(X_val, y_val),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "bidirectional_RNN")])

Saving TensorBoard log files to: model_logs/bidirectional_RNN/20230211-102004
Epoch 1/5
215/215 [==============================] - 15s 51ms/step - loss: 0.5093 - accuracy: 0.7428 - val_loss: 0.4607 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3156 - accuracy: 0.8723 - val_loss: 0.5050 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.2154 - accuracy: 0.9155 - val_loss: 0.5659 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.1525 - accuracy: 0.9458 - val_loss: 0.6255 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1063 - accuracy: 0.9616 - val_loss: 0.6747 - val_accuracy: 0.7717


In [60]:
# Make predictions with bidirectional RNN on validation data
model_4_pred_probs = model_4.predict(X_val)
model_4_pred_probs[:10]

24/24 [==============================] - 1s 5ms/step


array([[0.072896  ],
       [0.73730356],
       [0.99883014],
       [0.13129613],
       [0.00366358],
       [0.9934748 ],
       [0.95680064],
       [0.99935526],
       [0.99941397],
       [0.3122196 ]], dtype=float32)

In [61]:
# Convert prediction probabilities to labels
model_4_pred = tf.squeeze(tf.round(model_4_pred_probs))
model_4_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [63]:
# Calculate bidirection model results
model_4_results = calculate_results(y_val, model_4_pred)
model_4_results

{'accuracy': 77.16535433070865,
 'precision': 0.7713581681518764,
 'recall': 0.7716535433070866,
 'f1': 0.7709684019624822}